<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/Train_Accidents_Pred_road.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [14]:
import pandas as pd

#【祝日】
holi_df = pd.read_csv("./drive/My Drive/TrainAccident/holiday.csv")
holi_df["date"] = holi_df['date'].astype('str')
print(holi_df.head(5))

#【土日】
donichi_df = pd.read_csv("./drive/My Drive/TrainAccident/donichi.csv",index_col=0)
donichi_df["date"] = donichi_df['date'].astype('str')
donichi_list = donichi_df['date'].values.tolist()
print(donichi_df.head(5))

#【気温】
temp_df = pd.read_csv("./drive/My Drive/TrainAccident/temp.csv")
temp_df = temp_df.dropna(subset=["max_temp"])
temp_df = temp_df.dropna(subset=["min_temp"])
temp_df["date"] = temp_df['date'].astype('str')

#項目追加
temp_df = temp_df.assign(diff_temp=temp_df['max_temp'].astype(float)-temp_df['min_temp'].astype(float))
temp_df = temp_df.assign(max_temp_std=(temp_df["max_temp"] - temp_df["max_temp"].min()) / (temp_df["max_temp"].max() - temp_df["max_temp"].min()))
temp_df = temp_df.assign(min_temp_std=(temp_df["min_temp"] - temp_df["min_temp"].min()) / (temp_df["min_temp"].max() - temp_df["min_temp"].min()))
temp_df = temp_df.assign(diff_temp_std=(temp_df["diff_temp"] - temp_df["diff_temp"].min()) / (temp_df["diff_temp"].max() - temp_df["diff_temp"].min()))
print(temp_df.head(5))

#【日照時間】
sun_df = pd.read_csv("./drive/My Drive/TrainAccident/suntime.csv")
sun_df = sun_df.dropna(subset=["suntime"])
sun_df["date"] = sun_df['date'].astype('str')

#項目追加
sun_df = sun_df.assign(suntime_std=(sun_df["suntime"] - sun_df["suntime"].min()) / (sun_df["suntime"].max() - sun_df["suntime"].min()))
print(sun_df.head(5))

#【降水量/気圧】
rain_pres_df = pd.read_csv("./drive/My Drive/TrainAccident/rain_pres.csv")
rain_pres_df = rain_pres_df.drop('dmy',axis=1)
rain_pres_df["date"] = rain_pres_df['date'].astype('str')

#項目追加
rain_pres_df = rain_pres_df.assign(rain_std=(rain_pres_df["rain"] - rain_pres_df["rain"].min()) / (rain_pres_df["rain"].max() - rain_pres_df["rain"].min()))
rain_pres_df = rain_pres_df.assign(pres_std=(rain_pres_df["pres"] - rain_pres_df["pres"].min()) / (rain_pres_df["pres"].max() - rain_pres_df["pres"].min()))
print(rain_pres_df.head(5))

         date  holiday_flg
0  2010/01/01            1
1  2010/01/11            1
2  2010/02/11            1
3  2010/03/21            1
4  2010/03/22            1
          date  yobi
1   2010/01/02     5
2   2010/01/03     6
8   2010/01/09     5
9   2010/01/10     6
15  2010/01/16     5
         date  max_temp  min_temp  ...  max_temp_std  min_temp_std  diff_temp_std
0  2010/01/01       9.5       0.1  ...      0.163265      0.079618       0.451087
1  2010/01/02      10.1       0.7  ...      0.180758      0.098726       0.451087
2  2010/01/03       9.5       2.1  ...      0.163265      0.143312       0.342391
3  2010/01/04      10.2       4.0  ...      0.183673      0.203822       0.277174
4  2010/01/05      11.9       2.7  ...      0.233236      0.162420       0.440217

[5 rows x 7 columns]
         date  suntime  suntime_std
0  2010/01/01      9.2     0.676471
1  2010/01/02      9.0     0.661765
2  2010/01/03      6.6     0.485294
3  2010/01/04      5.2     0.382353
4  2010/01/05     

In [0]:
import numpy as np
import pandas as pd
from datetime import date, timedelta

#日付データ作成
dates_df = pd.DataFrame({"date": [pd.to_datetime("2019-08-22") + timedelta(days=i) for i in range(2)]},dtype="object")
date_list = dates_df.values.tolist()

#デフォルトでは小数点が付いてしまう為、int型に変換
time_cate = np.eye(6).astype(int)
yobi_cate = np.eye(7).astype(int)
holi_cate = np.eye(1).astype(int)
ninzu_cate = np.eye(5).astype(int)

wk_list = []
for wk in date_list:
  for wk2 in time_cate:
    for wk3 in yobi_cate:
      for wk4 in holi_cate:
        for wk5 in ninzu_cate:
          list_parts = np.concatenate((wk,wk2,wk3,wk4,wk5), axis=None)
          wk_list.append(list_parts.tolist())

col=["date",
     "comp_time","day_time","home_time","midnight_time","morning_time","night_time",
     "yobi_0","yobi_1","yobi_2","yobi_3","yobi_4","yobi_5","yobi_6",
     "holiday_flg",
     "ninzu_1","ninzu_2","ninzu_3","ninzu_4","ninzu_5"]

test_df = pd.DataFrame(data=wk_list,columns=col)

test_df['date'] = test_df['date'].astype('str')
test_df['date'] = test_df['date'].str.replace('-', '/')

#土日
test_df.loc[(test_df['date'].str[5:10].isin(donichi_list)) ,"holiday_flg"] = 1

#祝日
Special_holiday_list = ["08/13","08/14","08/15","08/16","12/29","12/30","12/31"]
test_df.loc[(test_df['date'].str[5:10].isin(Special_holiday_list)) ,"holiday_flg"] = 1

test_df.loc[(test_df['holiday_flg'].isnull()) ,"holiday_flg"] = 0

#日付削除
test_df2 = test_df.drop(["date"],axis=1)

In [33]:
v_max_temp = 30
v_min_temp = 25
v_diff_temp = v_max_temp - v_min_temp
v_sun = 7
v_rain = 10
v_pres = 1000

test_df2 = test_df2.assign(max_temp=temp_df[(temp_df["max_temp"] <= v_max_temp + 0.2) & (temp_df["max_temp"] >= v_max_temp - 0.2)]["max_temp_std"].mean())
test_df2 = test_df2.assign(min_temp=temp_df[(temp_df["min_temp"] <= v_min_temp + 0.2) & (temp_df["min_temp"] >= v_min_temp - 0.2)]["min_temp_std"].mean())
test_df2 = test_df2.assign(diff_temp=temp_df[(temp_df["diff_temp"] <= v_diff_temp + 0.2) & (temp_df["diff_temp"] >= v_diff_temp - 0.2)]["diff_temp_std"].mean())
test_df2 = test_df2.assign(suntime_temp=sun_df[(sun_df["suntime"] <= v_sun + 0.5) & (sun_df["suntime"] >= v_sun - 0.5)]["suntime_std"].mean())
test_df2 = test_df2.assign(rain=rain_pres_df[(rain_pres_df["rain"] <= v_rain + 1) & (rain_pres_df["rain"] >= v_rain - 1)]["rain_std"].mean())
test_df2 = test_df2.assign(pres=rain_pres_df[(rain_pres_df["pres"] <= v_pres + 1) & (rain_pres_df["pres"] >= v_pres - 1)]["pres_std"].mean())

test_df2.head(5)

,comp_time,day_time,home_time,midnight_time,morning_time,night_time,yobi_0,yobi_1,yobi_2,yobi_3,yobi_4,yobi_5,yobi_6,holiday_flg,ninzu_1,ninzu_2,ninzu_3,ninzu_4,ninzu_5,max_temp,min_temp,diff_temp,suntime_temp,rain,pres
0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0.760586,0.872551,0.212397,0.516063,0.0558,0.420467
1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0.760586,0.872551,0.212397,0.516063,0.0558,0.420467
2,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0.760586,0.872551,0.212397,0.516063,0.0558,0.420467
3,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0.760586,0.872551,0.212397,0.516063,0.0558,0.420467
4,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0.760586,0.872551,0.212397,0.516063,0.0558,0.420467


In [0]:
#保存したモデルをロードする(RFC)
import pickle
import pandas as pd

filename = './drive/My Drive/TrainAccident/Accident_RFC_model.sav'
loaded_RFC_model = pickle.load(open(filename, 'rb'))

In [34]:
# 評価する
y_pred = loaded_RFC_model.predict(test_df2)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,